In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
# csv files for answer responses
gdrive_path = "/Users/dorislee/Google Drive/Turn/user_study/evaluation_study/transcription_and_analysis/"

In [4]:
user_task =  pd.read_csv(gdrive_path+"UserTaskAssignment.csv")

In [5]:
%load_ext autoreload
%autoreload 2
from paper_visualization import *

In [6]:
def dcg(r, k, method=0):
    # alternative formulation of DCG places stronger emphasis on retrieving relevant documents
    r = np.asfarray(r)[:k]
    val = 0
    for i in range(1,len(r)):
        val+= (2**r[i]-1) / np.log2(i+1)
    return val

def ndcg(dcg_ground_truth,r, k):
    return dcg(r, k) / sum(dcg_ground_truth)

In [7]:
def compute_ndcg_rankings(user_ranking_filename,gt_ranking_filename):#,k_lst = [1,2,3,5,7]):

    user_rank = pd.read_csv(gdrive_path+user_ranking_filename)
    ground_truth_rank = pd.read_csv(gt_ranking_filename,index_col=0)
    k = len(ground_truth_rank["attribute"])
    ground_truth_cramerV_rank = list(ground_truth_rank["cramerV"].rank(ascending=False)) 
    #print ground_truth_cramerV_rank
    if "Police" in user_ranking_filename: 
        task_name = "Task1"
    else: 
        task_name = "Task2"
    #highest cramer's V corresponds to most correlated, corresponds to rank 1
    user_rank = user_rank.merge(user_task).drop("Note",axis=1)
    user_rankings=[]
    #Putting user rankings into appropriate data structures
    for row in user_rank.iterrows():
        ranking = []
        for attr in ground_truth_rank.attribute:
            ranking.append(row[1][attr])
        task = row[1][task_name]
        user_rankings.append([task,ranking])
    #Computing ndcg ranking
    ndcg_data = []
    for trial in user_rankings:
        ndcg_d = []
        #print trial
        ndcg_d.append(trial[0])
        #for k in k_lst:
        ndcg_d.append(ndcg(ground_truth_cramerV_rank,trial[1],k))
#             print trial[1]
        ndcg_data.append(ndcg_d)
#    print ndcg_data
    NDCG_lst = ["NDCG@"+str(k)]
    ndcg_result = pd.DataFrame(ndcg_data,columns=["Task"]+NDCG_lst)
    return ndcg_result.groupby("Task",as_index=False).mean()

In [10]:
user_ranking_filename = "Ranking(Police).csv"
gt_ranking_filename = "ct_police_ground_truth_correlation.csv"
user_rank = pd.read_csv(gdrive_path+user_ranking_filename)
ground_truth_rank = pd.read_csv(gt_ranking_filename,index_col=0)
k = len(ground_truth_rank["attribute"])
ground_truth_cramerV_rank = list(ground_truth_rank["cramerV"].rank(ascending=False)) 
#print ground_truth_cramerV_rank
if "Police" in user_ranking_filename: 
    task_name = "Task1"
else: 
    task_name = "Task2"
#highest cramer's V corresponds to most correlated, corresponds to rank 1
user_rank = user_rank.merge(user_task).drop("Note",axis=1)
user_rankings=[]
#Putting user rankings into appropriate data structures
for row in user_rank.iterrows():
    ranking = []
    for attr in ground_truth_rank.attribute:
        ranking.append(row[1][attr])
    task = row[1][task_name]
    user_rankings.append([task,ranking])
#Computing ndcg ranking
ndcg_data = []
for trial in user_rankings:
    ndcg_d = []
    #print trial
    ndcg_d.append(trial[0])
    #for k in k_lst:
    ndcg_d.append(ndcg(ground_truth_cramerV_rank,trial[1],k))
#             print trial[1]
    ndcg_data.append(ndcg_d)
#    print ndcg_data
NDCG_lst = ["NDCG@"+str(k)]
ndcg_result = pd.DataFrame(ndcg_data,columns=["Task"]+NDCG_lst)

In [29]:
ndcg_result=[["Police"]+list(compute_ndcg_rankings("Ranking(Police).csv","ct_police_ground_truth_correlation.csv")["NDCG@7"]),
["Autism"]+list(compute_ndcg_rankings("Ranking(Autism).csv","autism_ground_truth_correlation.csv")["NDCG@10"])]

In [31]:
print (T.tabulate(ndcg_result,headers=['Dataset','\system','Cluster','BFS'],tablefmt='latex', floatfmt=".2f"))

\begin{tabular}{lrrr}
\hline
 Dataset   &   \system &   Cluster &   BFS \\
\hline
 Police    &      0.63 &      0.45 &  0.84 \\
 Autism    &      0.50 &      0.35 &  0.24 \\
\hline
\end{tabular}


- NDCG@1 is 0 because if the first-ranked attribute is not retreived, then score would be 0.
- This makes sense, our algo does better than kmeans (2) and level-wise BFS (3), except for NDCG@3 A2 for some reason did very well.
- NDCG should be @10 because all users read through and ranked all 10 dashboards.